In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reset -sf

In [3]:
import importlib
import Deproject_v1_0 
from Deproject_test import *
from Deproject_plots import DM_plt_prefs
DM_plt_prefs()
import matplotlib.pyplot as plt
from scipy import sparse as scisp
from scipy.optimize import approx_fprime
import os
import cProfile
import numpy as np
from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle
from timeit import default_timer as timer

from astropy.io.ascii import read as tableread
import astropy.units as u
import astropy.coordinates as coord

coord.Galactic()
%cd ~/Documents/DeprojectionProject/

/home/mikkola/Documents/DeprojectionProject


In [4]:
%cd ~/Documents/faststats/faststats/
from fkde import *
def kdeplot(x,y,grid,no_c,bw_a):
    Z,ax = fastkde(x,y,gridsize=grid,nocorrelation=no_c,adjust=bw_a)#,weights,adjust)
    xlen,ylen = np.shape(Z)
    X,Y = np.meshgrid(np.linspace(ax[0],ax[1],xlen),np.linspace(ax[0],ax[1],ylen))
    return X,Y,Z
%cd ~/Documents/DeprojectionProject/

/home/mikkola/Documents/faststats/faststats
/home/mikkola/Documents/DeprojectionProject


In [ ]:
def rot(x,y,deg):
    xf = x.flatten(); xf = xf.reshape(len(xf),1)
    yf = y.flatten(); yf = yf.reshape(len(yf),1)
    pos = np.hstack((xf,yf))
    
    theta = np.radians(deg)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    pos = np.dot(pos, R.T)
    
    x = pos[:,0].reshape(x.shape)
    y = pos[:,1].reshape(y.shape)
    return x,y

def f(x,y,z,lim):
    x,y = rot(x,y,45)
    val = (lim-np.abs(x))**4 + (lim-np.abs(y))**4 + (lim-np.abs(z))**4
    return val/np.sum(val)


def pyramid_sample(N_sample,r,c):
    # Velocities
    xc,yc,zc = c
    N = 100
    X = np.linspace(-r,r,N)
    Y = np.linspace(-r,r,N)
    Z = np.linspace(-r,r,N)
    
    XX = X.reshape(1,N,1)*np.ones((N,1,N))
    YY = Y.reshape(N,1,1)*np.ones((1,N,N))
    ZZ = Z.reshape(1,1,N)*np.ones((N,N,1))
    
    P = f(XX,YY,ZZ,r+np.max(c))
    
    rrind = np.ravel_multi_index(np.indices(XX.shape), XX.shape).ravel()
    prob = np.ravel(P)
    smp = np.random.choice(rrind, size=N_sample, p=prob)
    smpx, smpy, smpz = np.asarray(np.unravel_index(smp, XX.shape))
    
    XX = XX + xc
    ZZ = YY + yc
    YY = ZZ + zc
    
    vxvals = XX[smpx, smpy, smpz]
    vyvals = YY[smpx, smpy, smpz]
    vzvals = ZZ[smpx, smpy, smpz]
    
    # Positions
    xmax, ymax, zmax = np.array([100,100,100])/np.sqrt(3)    
    xmin, ymin, zmin = -xmax,-ymax,-zmax
    
    xvals = (np.random.rand(N_sample) * (xmax - xmin) + xmin)
    yvals = (np.random.rand(N_sample) * (ymax - ymin) + ymin)
    zvals = (np.random.rand(N_sample) * (zmax - zmin) + zmin)
    return xvals, yvals, zvals, vxvals, vyvals, vzvals

## Generate gaussian sample

In [ ]:
N = 5000 #Number of stars we want to use in our sample
sample, vx, vy, vz = Deproject_v1_0.model_sample(N)

In [ ]:
pvals, rhatvals = Deproject_v1_0.calc_p_rhat(sample)
sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)

## Generate pyramid-like sample from distribution

In [ ]:
np.random.seed(0)
x, y, z, vx, vy, vz = pyramid_sample(int(1e4),100, c=(0,0,0))

In [ ]:
x,y,z          = (x,y,z)* u.pc
vx,vy,vz       = (vx,vy,vz) * (u.km / u.s)

In [ ]:
# We use Astropy's coord class which makes it easy to keep track of units and conversions

sample = coord.Galactic(u=x,
                         v=y, 
                         w=z,
                         U=vx,
                         V=vy,
                         W=vz,
                         representation_type=coord.CartesianRepresentation,
                         differential_type=coord.CartesianDifferential)
sample.set_representation_cls(coord.SphericalRepresentation, coord.SphericalCosLatDifferential)



In [ ]:
pvals, rhatvals = Deproject_v1_0.calc_p_rhat(sample)
sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)

## Generate  fake Galactic samples

From simulated galaxy:

Loading data

In [ ]:
N = 5000 #Number of stars we want to use in our sample
dat = np.load('DATA/simgal.npy')

Select a sample around with radius 2 kpc around 8 kpc

In [ ]:
x,y,z,vx,vy,vz = dat
R = np.sqrt((x-8)**2 + y**2)
snbh = R < 2
dat = dat[:,snbh]
I = np.random.randint(low=0,high=np.sum(snbh),size=N)

x,y,z,vx,vy,vz = dat[:,I]

# Center x, y and vy, so they look like they're relative to the LSR
x = x - np.median(x)
y = y - np.median(y)
vy = vy - np.median(vy)

Prepare for astropy

In [ ]:
x,y,z          = (x,y,z)* u.pc
vx,vy,vz       = (vx,vy,vz) * (u.km / u.s)

In [ ]:
# We use Astropy's coord class which makes it easy to keep track of units and conversions

sample = coord.Galactic(u=x,
                         v=y, 
                         w=z,
                         U=vx,
                         V=vy,
                         W=vz,
                         representation_type=coord.CartesianRepresentation,
                         differential_type=coord.CartesianDifferential)
sample.set_representation_cls(coord.SphericalRepresentation, coord.SphericalCosLatDifferential)


In [ ]:
pvals, rhatvals = Deproject_v1_0.calc_p_rhat(sample)
sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)

## Inspect sample

In [ ]:
plt.hist2d(np.array(sample.l),np.array(sample.b),100)
plt.show()

True distribution

In [ ]:
n = np.array([20,20,20])
dv = np.array([10,10,10])
vmin = np.array([-100,-100,-100])

In [ ]:
sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)

sigma = np.sqrt(sigma2)
phi0 = phi_guess(vmean0,sigma,vmin,dv,n)

sanity_check(pvals,rhatvals,phi0,vmin,dv,n)

In [ ]:
alpha_guess = 1e-8

v_guess = np.array([1,1,1])
disp_guess = np.array([22,17,30])

t0 = timer()
mxl, phi_all, fmin_it = max_L(alpha_guess, pvals, rhatvals, vmin, dv, n, v0_guess = v_guess, disp_guess = disp_guess)
time = str((timer() - t0)/60)
print(time + ' mins')

sanity_check(pvals,rhatvals,mxl,vmin,dv,n)

In [ ]:
dx, dy = dv[0], dv[1]
vxmin, vymin = vmin[0], vmin[1]
nx, ny = n[0], n[1]
vxmax, vymax = vxmin+nx*dx,vymin+ny*dy

x0, y0 = vxmin, vymin
x1, y1 = vxmax, vymax

xbins = np.arange(x0,x1+dx,dx)
ybins = np.arange(y0,y1+dy,dy)

xc = (xbins[1:]+xbins[:-1])/2
yc = (ybins[1:]+ybins[:-1])/2

[X,Y] = np.meshgrid(xc,yc);

fv = np.exp(mxl)

fvlog = np.log10(fv+1)
xl = 120
yl = 120
twodfv = np.sum(fv,axis=2)


X2,Y2,Z = kdeplot(np.array(vx),np.array(vy),(100,100),False,2)
# Z,xbins,ybins = np.histogram2d(np.array(vx),np.array(vy),20)
# xc = (xbins[1:]+xbins[:-1])/2
# yc = (ybins[1:]+ybins[:-1])/2

# [X2,Y2] = np.meshgrid(xc,yc);

f,ax = plt.subplots(1,3,sharex=True,sharey=True,figsize=(21,7),frameon=False)

ax[0].contourf(X2,Y2,Z.T,40,origin='lower',cmap = plt.cm.get_cmap('bone_r'),linestyles='solid')
ax[0].pcolormesh(X2,Y2,Z.T,cmap='bone_r')
ax[0].contour(X2,Y2,Z.T,10,origin='lower',colors='k',linewidths=0.5)
ax[0].set_xlim(-xl,xl)
ax[0].set_ylim(-yl,yl)
ax[0].add_patch(Rectangle((vmin[0],vmin[1]),n[0]*dv[0],n[0]*dv[0], fill=None,edgecolor='r',label='3D box size'))
ax[0].legend()
ax[0].set_title('Original fv')

ax[1].contourf(X,Y,twodfv.T,40,origin='lower',cmap = plt.cm.get_cmap('bone_r'),linestyles='solid')
ax[1].contour(X,Y,twodfv.T,10,origin='lower',colors='k',linewidths=0.5)
ax[1].set_xlim(-xl,xl)
ax[1].set_ylim(-yl,yl)
ax[1].add_patch(Rectangle((vmin[0],vmin[1]),n[0]*dv[0],n[0]*dv[0], fill=None,edgecolor='r',label='3D box size'))
ax[1].legend()
ax[1].set_title('Deprojected fv')

ax[2].contourf(X,Y,np.log10(twodfv.T),40,origin='lower',cmap = plt.cm.get_cmap('bone_r'),linestyles='solid')
ax[2].contour(X,Y,np.log10(twodfv.T),10,origin='lower',colors='k',linewidths=0.5)
ax[2].set_xlim(-xl,xl)
ax[2].set_ylim(-yl,yl)
ax[2].add_patch(Rectangle((vmin[0],vmin[1]),n[0]*dv[0],n[0]*dv[0], fill=None,edgecolor='r',label='3D box size'))
ax[2].legend()
ax[2].set_title('Deprojected log(fv)')

plt.show()

## Alternative sec_der

## Check eq 30 on 1d Gaussian

$$ \Xi_{\pmb{n}\pmb{l}} \propto -2\delta_{\pmb{n},\pmb{l}} + \delta_{\pmb{n},\pmb{l} + \pmb{\hat{e}}_i} + \delta_{\pmb{n},\pmb{l} - \pmb{\hat{e}}_i}$$

$$f(x) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2} \rightarrow \frac{1}{\sqrt{2\pi}} e^{-\frac{1}{2}x^2}$$

$$f'(x) =  -\frac{x}{\sqrt{2\pi}} e^{-\frac{1}{2}x^2}$$

$$f"(x) =  \frac{x^2 - 1}{\sqrt{2\pi}} e^{-\frac{1}{2}x^2}$$

In [ ]:
def f_gauss(x):
    return np.exp(-0.5*x**2) / np.sqrt(2*np.pi)

def f_gauss_bis(x):
    return (x**2 - 1)*f_gauss(x)



In [ ]:
x = np.linspace(-5,5,15)
vals = f_gauss(x)
sec_der_vals = f_gauss_bis(x)
xi_vals = vals[2:] + vals[:-2] - 2*vals[1:-1]

plt.figure()
plt.plot(x,vals,'b')
plt.plot(x,sec_der_vals,'r')
plt.plot(x[1:-1],xi_vals,'g')

In [ ]:
%matplotlib inline
from ipywidgets import *

def f_gauss(x):
    return np.exp(-0.5*x**2) / np.sqrt(2*np.pi)

def f_gauss_bis(x):
    return (x**2 - 1)*f_gauss(x)

def xi_vals(vals):
    return vals[2:] + vals[:-2] - 2*vals[1:-1]

def x_vals(N):
    return np.linspace(-5,5,N)

def plot_func(N):
    x = x_vals(int(N))

    plt.plot(x,f_gauss(x),'b')
    plt.plot(x,f_gauss_bis(x),'r')
    plt.plot(x[1:-1],xi_vals(f_gauss(x)),'g')
    plt.show()

interact(plot_func, N = widgets.FloatSlider(value=10,
                                               min=5,
                                               max=50,
                                               step=5))

In [ ]:
f,ax = plt.subplots(4,2,figsize=(8,16),frameon=False)

Z = np.log10(np.exp(mxl1) +1).sum(axis=2) 
ax[0,0].pcolormesh(Z.T,cmap='bone_r')
ax[0,0].contour(Z.T,10,origin='lower',colors='k',linewidths=0.5)
ax[0,0].set_ylabel('n = (%s,%s,%s)' % tuple([10]*3))
f.delaxes(ax[0,1])

rows = [[mxl2,mxl2b],[mxl3,mxl3b],[mxl4,mxl4b]]
for i, row in enumerate(rows):
    
    Z = np.log10(np.exp(row[0]) +1).sum(axis=2) 
    ax[i+1,0].pcolormesh(Z.T,cmap='bone_r')
    ax[i+1,0].contour(Z.T,10,origin='lower',colors='k',linewidths=0.5)
    ax[i+1,0].set_title('~' + duration[i][0])
    ax[i+1,0].set_ylabel('n = (%s,%s,%s)' % tuple([10*(2**(i+1))]*3))
    
    Z = np.log10(np.exp(row[1]) +1).sum(axis=2)
    ax[i+1,1].pcolormesh(Z.T,cmap='bone_r')
    ax[i+1,1].contour(Z.T,10,origin='lower',colors='k',linewidths=0.5)
    ax[i+1,1].set_title('~' + duration[i][1])


plt.savefig('multigrid.pdf',format='pdf')
# plotphi(np.log10(np.exp(mxl1) +1).sum(axis=2))
# plotphi(np.log10(np.exp(mxl2) +1).sum(axis=2))
# plotphi(np.log10(np.exp(mxl3) +1).sum(axis=2))
# plotphi(np.log10(np.exp(mxl4) +1).sum(axis=2))

In [ ]:
from scipy.ndimage import zoom

def params(N):
    n = N
    vmin = np.array([-150,-150,-150])
    vmax = np.array([150,150,150])
    dv = (vmax - vmin)/n
    return vmin, dv, n

def zoomed_mxl(mxl):
    phi0_guess = zoom(mxl, zoom=2, order=3)
    n = phi0_guess.shape
    return phi0_guess, n

alpha_guess = 1e-8
v_guess = np.array([1,1,1])
disp_guess = np.array([22,17,30])

vmin, dv, n = params(np.array([10,10,10]))


duration = []


t0 = timer()
# 1st
mxl1, phi_all, fmin_it = max_L(alpha_guess, pvals, rhatvals, vmin, dv, n, v0_guess = v_guess, disp_guess = disp_guess, disp=0)
time_spent = np.around((timer() - t0)/60, 3).astype(str) + ' mins'; t0 = timer()
duration.append(time_spent)

i_mxl2, N = zoomed_mxl(mxl1)
vmin, dv, n = params(N)


# 2nd
mxl2, phi_all, fmin_it = max_L(alpha_guess, pvals, rhatvals, vmin, dv, n, v0_guess = v_guess, disp_guess = disp_guess, phi0_guess=i_mxl2, disp=0)
time_spent = np.around((timer() - t0)/60, 3).astype(str) + ' mins'; t0 = timer()
duration.append(time_spent)

i_mxl3, N = zoomed_mxl(mxl2)
vmin, dv, n = params(N)


# 3rd
mxl3, phi_all, fmin_it = max_L(alpha_guess, pvals, rhatvals, vmin, dv, n, v0_guess = v_guess, disp_guess = disp_guess, phi0_guess=i_mxl3, disp=0)
time_spent = np.around((timer() - t0)/60, 3).astype(str) + ' mins'; t0 = timer()
duration.append(time_spent)

i_mxl4, N = zoomed_mxl(mxl3)
vmin, dv, n = params(N)


# duration.append([])
####4th
mxl4, phi_all, fmin_it = max_L(alpha_guess, pvals, rhatvals, vmin, dv, n, v0_guess = v_guess, disp_guess = disp_guess, phi0_guess=i_mxl4, disp=0)
time_spent = np.around((timer() - t0)/60, 3).astype(str) + ' mins'; t0 = timer()
duration.append(time_spent)

In [ ]:
f,ax = plt.subplots(4,1,figsize=(6,18),frameon=False)

mxl_list = [mxl1,mxl2,mxl3,mxl4]
for i, mxl in enumerate(mxl_list):
    vmin = np.array([-150,-150,-150])
    vmax = np.array([150,150,150])
    Z = np.log10(np.exp(mxl) +1).sum(axis=2) 
    
    ax[i].pcolormesh(np.linspace(vmin[0],vmax[0],Z.shape[0]),np.linspace(vmin[1],vmax[1],Z.shape[1]),Z.T,cmap='bone_r')
    ax[i].contour(Z.T,10,origin='lower', extent=(vmin[0],vmax[0],vmin[1],vmax[1]), colors='k',linewidths=0.5)
    ax[i].set_title('~' + duration[i] + ('\nn = (%s,%s,%s)' % mxl.shape), fontsize=14)
    ax[i].set_aspect('equal')
    ax[i].set_ylabel('$v_y$')
    ax[-1].set_xlabel('$v_x$')
plt.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:
N = 5000 #Number of stars we want to use in our sample
sample, vx, vy, vz = Deproject_v1_0.model_sample(N)

pvals, rhatvals = Deproject_v1_0.calc_p_rhat(sample)
sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)
sigma = np.sqrt(sigma2)

n = np.array([20,20,20])
dv = np.array([10,10,10])
vmin = np.array([-100,-100,-100])

phi0 = phi_guess(vmean0,sigma,vmin,dv,(nx+4,ny+4,nz+4))


alpha_guess = 1e-6
v_guess = np.array([1,1,1])
disp_guess = np.array([22,17,30])

Kvals = KvalsNumpyMethod(pvals, rhatvals, vmin, dv, n)
args = (Kvals, N, alpha_guess, dv, n, sigma2)
phi0r = np.ravel(phi0)

In [ ]:
mxl, phi_all = fmin_cg(get_negL, phi0r, fprime=get_grad_negL, gtol=1e-4, args=args, retall=True, disp=1, maxiter=1)

### What difference does padding with zeros or extrapolated values make for L and dL?

In [8]:
import inspect
import scipy.optimize as sopt

Get L and norm(grad_L) calculated with phi0 and compare it to a numerical approximation

In [ ]:
val_L_zero      = neg_L_calc(phi0r, 'zeros', Kvals, N, 1e-6, dv, n, sigma2)
norm_dL_zero    = np.linalg.norm(grad_neg_L_calc(phi0r, 'zeros', Kvals, N, 1e-6, dv, n, sigma2))
grad_check_zero = sopt.check_grad(neg_L_calc,grad_neg_L_calc,phi0r, 'zeros', Kvals, N, 1e-6, dv, n, sigma2)

Get L and norm(grad_L) calculated with phi0 and compare it to a numerical approximation

In [ ]:
val_L_extr      = neg_L_calc(phi0r, 'extr', Kvals, N, 1e-6, dv, n, sigma2)
norm_dL_extr    = np.linalg.norm(grad_neg_L_calc(phi0r, 'extr', Kvals, N, 1e-6, dv, n, sigma2))
grad_check_extr = sopt.check_grad(neg_L_calc,grad_neg_L_calc,phi0r, 'extr', Kvals, N, 1e-6, dv, n, sigma2)

### Comparison
Which lines are different in dL and L between the two codes?

L:

In [ ]:
_ = [print('1' + line) for line in lines_L_zero if (line not in lines_L_extr)]
_ = [print('2' + line) for line in lines_L_extr if (line not in lines_L_zero)]

dL:

In [ ]:
_ = [print('1' + line) for line in lines_dL_zero if (line not in lines_dL_extr)]
_ = [print('2' + line) for line in lines_dL_extr if (line not in lines_dL_zero)]

How does L compare?

In [ ]:
print(val_L_zero)
print(val_L_extr)

How does the norm of dL compare?

In [ ]:
print(norm_dL_zero)
print(norm_dL_extr)

How does dL compare to the numerical approximation?

In [ ]:
print(grad_check_zero)
print(grad_check_extr)
print(sopt.check_grad(neg_L_calc,grad_neg_L_calc,phi0r, 'extr', Kvals, N, 1e-6, dv, n, sigma2))

In [16]:
def xi_calc(phi, sigma2, dv):  
    if builtins.method == 'extr':
        phip = np.pad(phi,1)
        extrapolate_box(phip)
    elif builtins.method == 'zeros':
        phip = np.pad(phi,1)
    elif builtins.method == 'buffer':
        phip = np.copy(phi)
        phi = phi[1:-1, 1:-1, 1:-1]
        
    phi_fac = np.array([phip[0:-2, 1:-1, 1:-1] + phip[2:, 1:-1, 1:-1],
                        phip[1:-1, 0:-2, 1:-1] + phip[1:-1, 2:, 1:-1],
                        phip[1:-1, 1:-1, 0:-2] + phip[1:-1, 1:-1, 2:]])
    
    phi_arrx = (sigma2[0] / dv[0]**2) * (phi_fac[0] - 2 * phi)
    phi_arry = (sigma2[1] / dv[1]**2) * (phi_fac[1] - 2 * phi)
    phi_arrz = (sigma2[2] / dv[2]**2) * (phi_fac[2] - 2 * phi)
    
    phi_arr = phi_arrx + phi_arry + phi_arrz
    
    if builtins.method == 'extr':
        extrapolate_box(phi_arr)
    elif builtins.method == 'zeros' and builtins.called == 0:
        phi_arr = np.pad(phi_arr[1:-1, 1:-1, 1:-1],1)
        builtins.called += 1
        
    return phi_arr


def neg_L_calc(phi, *args):
    builtins.called = 0
    Kvals, N, alpha, dv, n, sigma2 = args
    nx, ny, nz = n
    
    if builtins.method == 'buffer':
        phi_unr = np.reshape(phi,(nx+4, ny+4, nz+4))
        phi = np.ravel(phi_unr[2:-2, 2:-2, 2:-2])
        t3 = get_t3(phi_unr[1:-1, 1:-1, 1:-1], n, sigma2, dv, alpha)
    else:
        phi_unr = np.reshape(phi,(nx, ny, nz))
        t3 = get_t3(phi_unr, n, sigma2, dv, alpha)
                             
    t2 = get_t2(phi)
    t1 = get_t1(t2, Kvals, N)
    t2 = t2.sum()
    
    L_tilde = t1 - t2 - t3
        
    neg_L = -1 * L_tilde
    return neg_L


def grad_neg_L_calc(phi, *args):
    builtins.called = 0
    Kvals, N, alpha, dv, n, sigma2 = args
    nx, ny, nz = n
    if builtins.method == 'buffer':
        phi_unr = np.reshape(phi,(nx+4, ny+4, nz+4))
        phi = np.ravel(phi_unr[2:-2, 2:-2, 2:-2])
        t3 = get_grad_t3(phi_unr, n, sigma2, dv, alpha)
    else:
        phi_unr = np.reshape(phi,(nx, ny, nz))
        t3 = get_grad_t3(phi_unr, n, sigma2, dv, alpha)
    

    t2 = get_grad_t2(phi)
    t1 = get_grad_t1(t2, Kvals, N)
    
    grad_L = np.asarray(t1 - t2 - t3).reshape(len(phi), )
    
    neg_grad_L = -1 * grad_L
    return neg_grad_L


def get_grad_t1(exphi_csc, Kvals, N):
    Kphi = exphi_csc.multiply(Kvals)

    Kphi_sum = Kphi_suminv = Kphi.sum(axis=1)

    Kphi_suminv[Kphi_sum.nonzero()] = 1 / Kphi_sum[Kphi_sum.nonzero()]

    K_term0 = Kphi.multiply(Kphi_suminv)
    K_term = K_term0.sum(axis=0)
    return K_term/N
    
    
def get_grad_t2(phi):
    exphir = np.exp(phi)

    exphi_coo = scisp.coo_matrix(exphir)
    exphi_csc = exphi_coo.tocsc()

    exphi_coo = scisp.coo_matrix(exphir)
    exphi_csc = exphi_coo.tocsc()
    return exphi_csc


def get_grad_t3(phi_unr, n, sigma2, dv, alpha):
    dvx, dvy, dvz = dv
    
    dphixhi = xi_calc(phi_unr, sigma2, dv)
    dphixhi = np.ravel(2*xi_calc(dphixhi, sigma2, dv))
    
    dphixhi_rav = dphixhi.reshape(1, dphixhi.size)
    dphixhi_coo = scisp.coo_matrix(dphixhi_rav)
    dphixhi_csc = dphixhi_coo.tocsc()
    return ((alpha * dvx * dvy * dvz) / 2) * dphixhi_csc


def get_t1(exphi_csc, Kvals, N):
    Kphi = Kvals.multiply(exphi_csc)
    Kphi_sum = Kphi.sum(axis=1)

    Kphi_sumlog = np.log(Kphi_sum[Kphi_sum != 0])
    Kphi_sum_tot = np.sum(Kphi_sumlog)
    return Kphi_sum_tot / N


def get_t2(phi):
    exphir = np.exp(phi)
    exphi_coo = scisp.coo_matrix(exphir)
    exphi_csc = exphi_coo.tocsc()
    return exphi_csc


def get_t3(phi_unr, n, sigma2, dv, alpha):
    dvx, dvy, dvz = dv
    phixhi = xi_calc(phi_unr, sigma2, dv) ** 2
        
    phixhi_sum = np.sum(phixhi)
    return ((alpha * dvx * dvy * dvz) / 2) * phixhi_sum

    

In [19]:
def get_args():
    N = 5000 #Number of stars we want to use in our sample
    sample, vx, vy, vz = Deproject_v1_0.model_sample(N)

    pvals, rhatvals = Deproject_v1_0.calc_p_rhat(sample)
    sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)
    sigma = np.sqrt(sigma2)

    n = np.array([6,6,6])
    dv = np.array([10,10,10])
    vmin = np.array([-100,-100,-100])

    alpha_guess = 1e-6
    v_guess = np.array([1,1,1])
    disp_guess = np.array([22,17,30])

    Kvals = KvalsNumpyMethod(pvals, rhatvals, vmin, dv, n)

    nx, ny, nz = n
    phi0 = phi_guess(vmean0,sigma,vmin,dv,(nx+4,ny+4,nz+4))
    if builtins.method == 'buffer':    
        phi0r = np.ravel(phi0)
    else:
        phi0r = np.ravel(phi0[2:-2, 2:-2, 2:-2])
    return phi0r, Kvals, N, alpha_guess, dv, n, sigma2

In [20]:
print('neg_L')
for method in ['zeros', 'extr', 'buffer']:
    builtins.method = method
    phi0r, Kvals, N, alpha_guess, dv, n, sigma2 = get_args()
    
    builtins.method = method
    val_L = neg_L_calc(phi0r, Kvals, N, alpha_guess, dv, n, sigma2)
    print(method + ': ' + str(val_L))

print('\n|grad_L|')
for method in ['zeros', 'extr', 'buffer']:
    builtins.method = method
    phi0r, Kvals, N, alpha_guess, dv, n, sigma2 = get_args()
    
    builtins.method = method
    norm_dL = np.linalg.norm(grad_neg_L_calc(phi0r, Kvals, N, alpha_guess, dv, n, sigma2))
    print(method + ': ' + str(norm_dL))

neg_L
zeros: 1.0665358619083867
extr: 1.1708693626955984
buffer: 1.100315698489635

|grad_L|
zeros: 0.033497112974967144
extr: 0.00685483257687065
buffer: 0.007645423902062214


In [21]:
print('\nGradient check')
for method in ['zeros', 'extr']:
    builtins.method = method
    phi0r, Kvals, N, alpha_guess, dv, n, sigma2 = get_args()
    gcheck = sopt.check_grad(neg_L_calc,grad_neg_L_calc,phi0r, Kvals, N,alpha_guess, dv, n, sigma2)
    print(method + ': ' + str(gcheck))


Gradient check
zeros: 1.825056364085722e-07
extr: 1.4041602411713587
